# save maf object
- pre-filter for depth of > 50, max_af < 0.01, filter == PASS

In [ ]:
library(future)
library(maftools)
library(dplyr)
library(data.table)
library(gdata)
plan("multicore", workers=15)
options(future.globals.maxSize=120000 * 1024^2)

## data loading 

In [ ]:
main_path <- "/expanse/lustre/projects/csd728/m7huang/somatic_filtering_20250528/all_samples/discovery/"

In [ ]:
mafs <- list.files(paste0(main_path, "maf_samples"), 
           full.names = TRUE)
head(mafs)

In [ ]:
sampleNames <- list.files(paste0(main_path, "maf_samples"))
sampleNames <- sub("\\.maf$", "", sampleNames)
head(sampleNames)

In [ ]:
filepath <- file.path(paste0(main_path, "maf_objs"))
filepath

## maf filtering

In [ ]:
# filters maf files and stores into maf_objs directory

filterMafObj <- function(maf, sampleName){
    mafObj <- read.maf(maf)
    
    rare <- subsetMaf(mafObj, query = ("t_depth >= 50"), mafObj = FALSE)
    # rare2 <- subset(rare, (!Variant_Classification %in% c("RNA", "Silent","3'UTR", "Intron", "Splice_Region", "5'UTR","3'Flank","5'Flank")))
    rare3 <- subset(rare, (MAX_AF < 0.01 | is.na(MAX_AF) | MAX_AF=="" | MAX_AF=="-" | MAX_AF=="."))
    rare4 <- subset(rare3, FILTER == "PASS")
    rare5 <- read.maf(rare4)
    
    fileName <- paste(sampleName, ".RData", sep = "")
    save(rare4, file = file.path(filepath, fileName))
}
mapply(filterMafObj, mafs, sampleNames)

## maf merging

In [ ]:
maf_files <- list.files(filepath, full.names = TRUE)
head(maf_files)

In [ ]:
#loads all maf objects under unique names for upcoming merge
readMafObj <- function(maf, sampleName){
    load(maf, envir = .GlobalEnv)
    mv(from = "rare4", to = sampleName, envir = .GlobalEnv)
}
mapply(readMafObj, maf_files, sampleNames)

In [ ]:
mafObjs <- mget(ls(pattern = "^Sample"))
length(mafObjs)

In [ ]:
merged_maf_obj <- merge_mafs(mafObjs)
merged_maf_obj

In [ ]:
save(merged_maf_obj, file = paste0(main_path, "merged_maf_obj_20250602.RData"))

# rare and rare damaging variants

In [ ]:
main_path <- "/expanse/lustre/projects/csd728/m7huang/somatic_filtering_20250528/all_samples/discovery/"
obj <- "merged_maf_obj_20250602.RData"

In [ ]:
load(paste0(main_path, obj))
disc <- merged_maf_obj

temp <- disc@data
temp$cohort <- "disc"
disc@data <- temp
head(disc@data)

In [ ]:
disc

In [ ]:
main_path <- "/expanse/lustre/projects/csd728/m7huang/somatic_filtering_20250528/all_samples/validation/"
obj <- "validation_merged_maf_obj_20250602.RData"
load(paste0(main_path, obj))
val <- merged_maf_obj

temp <- val@data
temp$cohort <- "val"
val@data <- temp
head(val@data)

In [ ]:
val

In [ ]:
rare <- disc
name <- "disc_20250721"

# rare <- val
# name <- "val_20250721"
rare

In [ ]:
table(rare@data$'Variant_Classification')

In [ ]:
rare <- subset(rare@data, 
               (!Variant_Classification %in% c("RNA", "Silent", "Splice_Site")))


In [ ]:
table(rare$'Variant_Classification')

In [ ]:
# rare <- merged_maf_obj@data
dim(rare)

## variant filtration

In [ ]:
#keep variants that are in gnomAD database 
rare_gnomad <- subset(rare, ((!is.na(MAX_AF) | !MAX_AF=="" | !MAX_AF=="-" | !MAX_AF==".")))

dim(rare)
dim(rare_gnomad)

In [ ]:
rare_novel <- subset(rare, ((is.na(MAX_AF) | MAX_AF=="" | MAX_AF=="-" | MAX_AF==".") & !(is.na(dbSNP_RS) | dbSNP_RS == "novel"|dbSNP_RS == "")))

dim(rare)
dim(rare_novel)

In [ ]:
rare_indels <- subset(rare, !(Variant_Type == "SNP") & (!(is.na(dbSNP_RS) | dbSNP_RS == "novel"|dbSNP_RS == "") | !(is.na(MAX_AF) | MAX_AF=="" | MAX_AF=="-" | MAX_AF==".")))

dim(rare)
dim(rare_indels)

In [ ]:
rare_merge1 <- rbind(rare_gnomad, rare_novel)
dim(rare_gnomad)
dim(rare_novel)
dim(rare_merge1)

rare_merge<-rbind(rare_merge1, rare_indels)
dim(rare_indels)
dim(rare_merge)

In [ ]:
# Remove duplicate rows (all columns)
library(dplyr)
rare_unique <- rare_merge %>% distinct()
dim(rare_unique)
head(rare_unique)

In [ ]:
rare_unique_maf <- read.maf(rare_unique)

plotmafSummary(maf = rare_unique_maf, rmOutlier = TRUE, 
               addStat = 'median', dashboard = TRUE, titvRaw = TRUE)
rare_unique_maf

## patient & gene level filtration

In [ ]:
#remove variants occurring in all patients (or some >% of patients) to remove false positives

df <- rare_unique
# get number of samples
(( n_Source_MAF <- df %>% pull(Tumor_Sample_Barcode) %>% unique() %>% length() ))

rare_vars <- df %>% 
    select(Tumor_Sample_Barcode, dbSNP_RS) %>% 
    group_by(dbSNP_RS) %>% 
    count() %>% # count how many samples the variant is present in
    ungroup() %>% 
    filter(n<(0.7*n_Source_MAF)) %>% # filter out variants present in 70%+ of samples
    pull(dbSNP_RS)

length(rare_vars)
head(rare_vars)

In [ ]:
rare_filtered <- subset(rare_unique, dbSNP_RS %in% rare_vars)
dim(rare_filtered)
head(rare_filtered)

rare_filtered_maf <- read.maf(rare_filtered)
rare_filtered_maf
plotmafSummary(maf = rare_filtered_maf, rmOutlier = TRUE, addStat = 'median', dashboard = TRUE, titvRaw = TRUE)

In [ ]:
flags <-c('TTN','MUC16','OBSCN','AHNAK2','SYNE1','FLG','MUC5B','DNAH17','PLEC',
          'DST','SYNE2','NEB','HSPG2','LAMA5','AHNAK','HMCN1', 'USH2A','DNAH11',
          'MACF1','MUC17','DNAH5','GPR98','FAT1','PKD1','MDN1','RNF213','RYR1',
          'DNAH2','DNAH3','DNAH8','DNAH1','DNAH9', 'ABCA13','APOB','SRRM2','CUBN',
          'SPTBN5','PKHD1','LRP2','FBN3','CDH23','DNAH10','FAT4','RYR3','PKHD1L1',
          'FAT2','CSMD1','PCNT', 'COL6A3','FRAS1','FCGBP','DNAH7','RP1L1','PCLO',
          'ZFHX3','COL7A1','LRP1B','FAT3','EPPK1','VPS13C','HRNR','MKI67','MYO15A',
          'STAB1','ZAN','UBR4','VPS13B','LAMA1','XIRP2','BSN','KMT2C','ALMS1','CELSR1',
          'TG','LAMA3','DYNC2H1','KMT2D','BRCA2','CMYA5','SACS', 'STAB2','AKAP13',
          'UTRN','VWF','VPS13D','ANK3','FREM2','PKD1L1','LAMA2','ABCA7','LRP1','ASPM',
          'MYOM2','PDE4DIP','TACC2','MUC2','TEP1','HELZ2','HERC2','ABCA4')
rare_filtered_no_flags <- subset(rare_filtered, !Hugo_Symbol %in% flags)

rare_filtered_no_flags_maf <- read.maf(rare_filtered_no_flags)
rare_filtered_no_flags_maf
plotmafSummary(maf = rare_filtered_no_flags_maf, rmOutlier = TRUE, addStat = 'median', dashboard = TRUE, titvRaw = TRUE)

In [ ]:
filter_genes <- getGeneSummary(rare_filtered_no_flags_maf)[,c('Hugo_Symbol', 'MutatedSamples')]
num_samples <- rare_filtered_no_flags_maf@data %>% pull(Tumor_Sample_Barcode) %>% unique() %>% length() 

print(num_samples)
head(filter_genes)

filter_genes <- subset(filter_genes, MutatedSamples > num_samples/2)$Hugo_Symbol
head(filter_genes)

In [ ]:
rare_final <- subset(rare_filtered_no_flags, !Hugo_Symbol %in% filter_genes)
rare_final_maf <- read.maf(rare_final)
rare_final_maf

write.mafSummary(rare_final_maf, 
                 basename = paste0("/expanse/lustre/projects/csd728/m7huang/somatic_filtering_20250528/",
                                  name))

plotmafSummary(maf = rare_final_maf, rmOutlier = TRUE, 
               addStat = 'median', dashboard = TRUE, titvRaw = TRUE)